# Lab 2: Hebbian Learning


### The Ubiquity of Associations
How do you learn to associate things that occur together?  \\
Peaches and ____________? \\
Cause and ____________? \\
Law and ____________? \\
Sooner or ____________? \\
Jack and ____________ went up ______ ____________ ? \\
 \
You can automatically fill in most or all of the blanks using associations you have learned in the past.  Associations are everywhere in cognition.  You learn to associate all the different properties of individual objects, all the objects that make up familiar scenes, and generally any groups of people, properties and/or things that tend to co-occur.  When you simply look at your laptop you know about its weight, texture, range of likely temperatures, approximately how long the battery will operate before dying, and lots of other associated details.  When you see only the top of a chair on the other side of a solid table you automatically infer details about the rest of the chair that is out of sight --  the top part of the chair that you see is strongly associated with the rest of a representation of a chair.  What do you expect to see in the waiting room of a doctor's office?  As each detail comes to mind it triggers associations to other details: chairs to sit in while you wait --> coffee table --> magazines --> Oprah magazine, Sports illustrated, The Economist --> cheap art on walls --> laminate flooring --> hand sanitizer --> clip boards with forms --> etc.  When you see lightning, what do you expect to hear?  Associations can build up in temporal proximity as well as spatial proximity.  Your stream of consciousness is partly built out of strong and weak associations.  How does the brain learn associations in the first place?  

### A Theory of Learning
Perhaps the most influential early theory in neuroscience is about learning associations, summarized as: "Neurons that fire together, wire together."  The theory was advanced in great detail by Donald Hebb in 1949, who stated, "The general idea is an old one, that any two cells or systems of cells that are repeatedly active at the same time will tend to become 'associated', so that activity in one facilitates activity in the other," and it is now called Hebbian Learning.  It is an elegant theory because it is both simple and has the potential to explain a huge amount of psychological and neuroscientific data.  And it is also a mechanistic theory that we can explore by building models.  

### Unsupervised Learning
Hebbian learning is a type of **unsupervised learning** because it is able to extract structure from data without using feedback.  There are no answers provided in unsupervised learning, only an acquired representation of structure present in the data.  In contrast, **supervised learning** usually involves a decision that can be correct or incorrect (e.g. Is this a picture of a dog or a cat?), and uses training and feedback to improve accuracty.  Subsequent labs will explore supervised learning in detail.     

### In this lab we will cover the following 
### concepts: 
Feature encoding \\
Correlation matrices \\
Simple Hebbian learning \\
Hebbian learning with pruning \\
Necker cubes \\

### tools: 
Matrix multiplication \\
Recurrent transfer mechanisms \\

In [ ]:
!pip install psyneulink


In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import psyneulink as pnl

# This couple of lines sets our color palette
import seaborn as sb
sb.palplot(sb.color_palette("RdBu_r",7))
sb.set_palette("RdBu_r")
sb.set_style("whitegrid")

matplotlib.colors.Colormap("RdBu_r")

print("Imports Successful")
print("This will be our color palette")

## Learning to Group Properties of Objects

Objects typically have multiple properties, such as size, shape, color, texture, density, temperature, etc.  It is useful to learn to group the different properties of an object together.  

In the following cell we begin by defining a set of features.  Our features are size (small, medium, large), color (red, blue, green), and shape (circle, rectangle, triangle).  When one of these features is present, it will be coded as a 1 and when it is absent it will be coded as a 0.  Next we will specify some objects that have subsets of the features.  As you can see in the code below a small red circle is coded as [1,0,0,1,0,0,1,0,0], and then abbreviated to src.

### Stimuli

In [ ]:
# define set of features
FeatureNames=['small','medium','large','red','yellow','blue','circle','rectangle','triangle']

# create a variable that corresponds to the size of our feature space
sizeF = len(FeatureNames)  

# define objects composed of features
small_red_circle = [1,0,0,1,0,0,1,0,0]
medium_yellow_rectangle = [0,1,0,0,1,0,0,1,0]
large_blue_triangle = [0,0,1,0,0,1,0,0,1]

# abbreviate the object names
src = small_red_circle
myr = medium_yellow_rectangle
lbt = large_blue_triangle

# Note that feature coding can be more elaborate, e.g. combining the basic feature... 
# e.g. Represent green as the activation of blue+yellow, and use rectangle+triangle for a house
small_green_house = [1,0,0,0,1,1,0,1,1] 
sgh = small_green_house

With our features defined, we can specify a collection of stimuli (sm_3_uniform) that will be presented to the model.  This collection of stimuli is created as a matrix.  Every row in the matrix is one object that will be presented.  Every column in the matrix is a feature (named in FeatureNames). Sm_3_uniform is a stimulus matrix containing three of each of the stimuli.

In [ ]:
sm_3_uniform=np.matrix([src,src,src,myr,myr,myr,lbt,lbt,lbt])
sm_3_uniform

You can also think about every feature as being represented by one artificial neuron.  There is a neuron that fires in response to "small" things and a neuron that fires to "medium" size things, etc: a total of 9 feature neurons.  

When we present a multi-feature stimulus to the system, we expect the relevant neurons to all fire. We want those neurons to become mathematically associated with each other, in roughly the same way that they would become physically associated with each other in the brain.  Later, if only one stimulus (e.g."red") is presented, we will see the associated neurons ("small" and "circle") fire as well.  We want the neurons that fire together to wire together.  

### Hebbian Model #1

To model this, we are going to look at all of the activations to all of the stimuli (conveniently, in this example the stimuli and corresponding patterns of activation are coded the exact same way with 0s and 1s).  Whenever the "red" neuron is active, what other neurons are active?  There is a useful statistic to measure linear relationships between variables; correlation.  Correlation coefficients are bounded between -1 and 1.  There are two important features of a correlation, sign (+/-) and strength.  When two variables always vary together in the same direction (like temperature and the volume of mercury) they are positively correlated.  When two variables vary in opposite directions (like temperature and layers of clothing worn) they are negatively correlated.  When two variables are completely independent, their correlation is 0.  The strength of the relationship is indicated by the magnitude of the absolute value of the correlation coefficient (in terms of strength -1=1; -0.5=0.5).  Look at the stimuli in the figure above.  Red always goes with circle -- these features have a correlation of 1.             

In the next cell we compute a correlation matrix for the features of all of our stimuli.  Rows and columns are the same set of feature neurons, and the values in the matrix are correlations between these neurons.  For example, the value in the first row ("small" neuron) and fifth column ("yellow" neuron) will tell us how often "small" and "yellow" fired at the same time.  (Remember that to inspect this value it is cor_mat_command[0,4] because Python indexing starts with 0 instead of 1.)

In [ ]:
# This portion of the cell creates the correlation matrix
cor_mat_command = np.corrcoef(sm_3_uniform, rowvar=False)
ub=np.amax(np.amax(np.abs(cor_mat_command)))
lb=-ub

# rowvar=False uses columns, rowvar=True uses rows
# It is worth noting that a matrix of correlation coefficients is symmetric across the principal diagonal.

# This portion of the cell visualizes the correlation matrix.
plt.figure()
plt.title('Lower Half of Correlation Matrix')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
mask_u =  np.triu(cor_mat_command)
tri_img_u = np.ma.array(cor_mat_command, mask=mask_u)
plt.imshow(tri_img_u,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.colorbar()
plt.show()

plt.figure()
plt.title('Upper Half of Correlation Matrix')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
mask_l =  np.tril(cor_mat_command)
tri_img_l = np.ma.array(cor_mat_command, mask=mask_l)
plt.imshow(tri_img_l,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.colorbar()
plt.show()

plt.figure()
plt.title('Full Correlation Matrix')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.imshow(cor_mat_command,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.colorbar()
plt.show()


The correlation matrix, cor_mat_command, effectively represents all of the connections between all of our neurons.  The principal diagonal (top left to bottom right) is the self-correlation; a neuron is perfectly correlated with itself, so it should be all 1's.  Except on the diagonal, every connection is depicted twice in this matrix.  For example, [1,2] is the same value as [2,1].  

The correlation matrix can be thought of as the probability that the firing of two neurons will coincide, based on the frequency that their firing coincided in the past. The rows and columns of the matrix represent the features, ordered as they are in our feature list. Each entry represents the connection between a pair of neurons.  One can think of the entries in the correlation matrix as directional probability, where the magnitude gives the strength of association, while the sign gives positive or negative association. 

Positive values mean that the neurons tend to both 1) fire together and 2) be inactive at the same time.  Negative values mean that when one of the neurons is firing, the other one tends to be inactive (and vice a versa).  Values close to 0 mean that the neurons do not exhibit a linear relationship in their patterns of activity.  (Note that correlations only detect linear relationships.  If, for example, the variables had a perfectly U shaped relationship that would produce a correlation of 0.)      

Once the connections between all the feature neurons have been established from learning, these connections can be useful for performing pattern completion.  For example, if the model is given an object that is a triangle, it can use the connections to infer that this object is probably also large and blue.      

In [ ]:
# incomplete stimulus for the trained model, providing only information about shape 
triangle = [0,0,0,0,0,0,0,0,1]

# use the triangle stimulus to activate all neurons connected to the triangle neuron
active_mat = np.dot(np.diag(triangle),cor_mat_command)

# sum the activations 
active_mat.sum(axis=0)

## the previous two steps can be also be accomplished more compactly using 
active_mat=np.matmul(triangle,cor_mat_command)
ub=np.amax(np.amax(np.abs(active_mat)))
lb=-ub

# This plot visualizes the activations from a triangular stimulus. The output is an array,
# and its values are mapped to the principal diagonal of this image. All off-diagonal elements are zero.
plt.figure()
plt.imshow(np.diag(active_mat),cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Activations of a Triangular Stimulus with Cor_mat_command')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

# This plot shows the same values but as a line plot.
plt.figure()
plt.stem(active_mat)
plt.title('Activations of a Triangular Stimulus with Cor_mat_command')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()



## Exercises

If you have not already done so, create your homework Jupyter notebook now.  This week's lab includes a series of Lab Exercises that you should **complete in lab and include as answers at the beginning of your Homework** file and submission on Blackboard.  You should also try to start and complete as many of the homework questions as you can while still in lab, where you have immediate access to help from your classmates and lab instructor.      

### Accumulating Evidence 

In the previous example we made an over-simplifying assumption that the stimuli were all present at the same time.  But most learning doesn't happen at a single moment in time, it builds up over experience.  We also computed correlations that include negative values, but the most basic implementation of "fire together wire together" should only detect simultaneous firing, not simultaneous absences of firing.  So next we can build a model that accumulates evidence over time and only detects neurons firing together.  

### Hebbian Model #2

We start with an empty connectivity matrix, full of 0's to indicate that no neurons are wired together yet.  When two neurons fire at the same time we'll add a weight of 0.1 to the matrix.

In [ ]:
# Create a new, all zeros correlation matrix
cor_mat_manual = np.zeros((sizeF,sizeF)) 

# number of stimuli, np.shape returns the number of rows and columns for an array.  The first value [0] is rows.    
sizeS = np.shape(sm_3_uniform)[0] 

for i in range(sizeS):  # progress through all stimuli
    stim = sm_3_uniform[i]
    for x in range(sizeF): 
        for y in range(sizeF):
            cor_mat_manual[x,y]=cor_mat_manual[x,y]+(stim[0,x]*stim[0,y]*.1)

ub=np.amax(np.amax(np.abs(cor_mat_manual)))
lb=-ub            
           
plt.figure()
plt.title('Full Correlation Matrix')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.imshow(cor_mat_manual,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.colorbar()
plt.show()

### Exercise 1

Suppose you want to know what a Hebbian Model #1 learner would associate and infer if they had seen, in their entire experience 10 small red circles, 7 medium yellow rectangles, and 4 large blue triangles --  Then, for the first time, they are exposed to the color purple.  Create a set of stimuli and then a correlation matrix that fit this situation.  How do you code purple as a stimulus?  What output do you get from testing the trained model on an input of purple?  Explain how you interpret this output.

In the example above, we used 3 loops to update all of the values in cor_mat.  You should reason through how and why this works by considering a few specific examples.  When i = 0, what are the values of stim?  When x and y equal 0, what number will be updated in cor_mat?

There is a more compact way to accomplish the same result as the 3 loops above, using the transpose function and a single loop.    

### Exercise 2a

You can transpose an array (turn rows into columns, and columns into rows) by adding .T to the array:

```{python}
stim_mat[0,:].T
```

And you can do matrix multiplication of two arrays with the appropriate dimensionality using @:

```{python}
stim_mat[0,:].T@stim_mat[0,:] 
```

Now create a single loop that progresses through the stimuli in stim_mat and creates the same resulting cor_mat2 values as above.  


### Exercise 2b

Matrix multiplication can simplify the code even more, eliminating the need for any loops.  Use the transpose function .T and matrix multiplication function @ to matrix multiply stim_mat by itself transposed.  As a final step, multiply this entire function by a value that will reproduce the same matrix values in cor_mat above.     

In the following cells we are going to progress through a series of steps that unpack some of the functionality in PsyNeuLink Hebbian learning, using Python examples.  Similar to Exercise 1, in the next cell we are going to create a matrix of stimuli that includes 100 small red circles, 70 medium yellow rectangles, and 40 large blue triangles.  Then we will use transpose and matrix multiplication to build a matrix of connection strengths.    

In [ ]:
sm_nonuniform=np.array([100*np.array(src),70*np.array(myr),40*np.array(lbt)])
cor_mat_nonuniform=0.1*sm_nonuniform.T@sm_nonuniform

ub=np.amax(np.amax(np.abs(cor_mat_nonuniform)))
lb=-ub
# This portion of the code visualizes the correlation matrix
plt.figure()
plt.imshow(cor_mat_nonuniform,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Correlation Matrix Created by Matrix Multiplication for Nonuniform Stimuli')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()


Now we can present a purple stimulus to this model, and see what output it generates.    

In [ ]:
purple=np.array([0,0,0,1,0,1,0,0,0])
purple_act=cor_mat_nonuniform@purple.T

#This section of code visualizes the input: purple

plt.figure()
plt.stem(purple)
plt.title('Input: Purple Stimulus')
plt.xticks(np.arange(0,sizeF))
plt.xlabel('Feature Index')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero value for visibility

purple_im=np.zeros((sizeF,sizeF))
np.fill_diagonal(purple_im,purple)
ub=np.amax(np.amax(np.abs(purple_im)))
lb=-ub

plt.figure()
plt.imshow(purple_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

#This section of code visualizes the output

purple_act_im=np.zeros((sizeF,sizeF))
np.fill_diagonal(purple_act_im,purple_act)
ub=np.amax(np.amax(np.abs(purple_act_im)))
lb=-ub

plt.figure()
plt.stem(purple_act)
plt.title('Activation from Purple Stimulus with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero.

plt.figure()
plt.imshow(purple_act_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

# To interpret the output, remember the feature list:
# FeatureNames=['small','medium','large','red','yellow','blue','circle','rectangle','triangle']

The model associated the color purple mostly with small red circle,  slightly with large blue triangle, and not at all with medium yellow rectangle.  This makes sense because the model was trained with more exposure to small red circles.  Purple activated red and it's strongest associates of small and circle.  Purple also activated blue and it's associates of large and triangle.     

### Bounding Output

It is often a good idea to set boundaries on the values of neuron activity in a model.  Most neurons can only physically fire a maximum of a few hundred times per second, and tend to fire closer to the order of around 10 times per second.  Even when we take the liberty of ignoring physical constraints on individual neurons (or even if we assume the unit we are modeling is composed of thousands of neurons and could, in principle, cumulatively send hundreds of thousands of action potentials per second), unbounded values that tend toward +/- infinity cause practical problems for many functions.  In the following cells we will explore some ways to regularize our model outputs.  

In [ ]:
# We define a new stimulus, then transpose it and matrix multiply by cor_mat. 
new_stim=np.array([1,0,0,1,0,0,0,0,0])
new_stim_act=cor_mat_nonuniform@new_stim.T

#This section of code visualizes the input: new_stim

plt.figure()
plt.stem(new_stim)
plt.title('Input: New_stim')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero

new_stim_im=np.zeros((sizeF,sizeF))
np.fill_diagonal(new_stim_im,new_stim)
ub=np.amax(np.amax(np.abs(new_stim_im)))
lb=-ub

plt.figure()
plt.imshow(new_stim_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Activations of New_stim')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

#This section of code visualizes the output

new_stim_act_im=np.zeros((sizeF,sizeF))
np.fill_diagonal(new_stim_act_im,new_stim_act)
ub=np.amax(np.amax(np.abs(new_stim_act)))
lb=-ub

plt.figure()
plt.stem(new_stim_act)
plt.title('Activation from New_stim with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero

plt.figure()
plt.imshow(new_stim_act_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Activation from New_stim with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

# To interpret the output, remember the feature list:
# FeatureNames=['small','medium','large','red','yellow','blue','circle','rectangle','triangle']

In [ ]:
# Dividing an array by its maximum value sets 1 as the maximum value.
new_stim_act=(new_stim_act/max(new_stim_act))

#This section of code visualizes the normalized output

plt.figure()
plt.stem(new_stim_act)
plt.title('Activation from Normalized New_stim with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

In [ ]:
# Initially, this bolsters the values to 1, rather than limiting them. However, with sufficiently repeated stimuli, we know the matrix
# will grow infinitely.

# To find the max value and then regularize the mat itself:

print(np.amax(cor_mat_nonuniform))
ub=np.amax(np.amax(np.abs(cor_mat_nonuniform)))
lb=-ub

# This portion of the code visualizes the correlation matrix
plt.figure()
plt.imshow(cor_mat_nonuniform,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Correlation Matrix Created by Matrix Multiplication for Nonuniform Stimuli')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

cor_mat_nonuniform=(np.amax(cor_mat_nonuniform)**-1)*cor_mat_nonuniform
ub=np.amax(np.amax(np.abs(cor_mat_nonuniform)))
lb=-ub

# This portion of the code visualizes the normalized correlation matrix
plt.figure()
plt.imshow(cor_mat_nonuniform,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Correlation Matrix Created by Matrix Multiplication for Nonuniform Stimuli')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()


# Note how the absolute values change but the relative values are identical

In [ ]:
# now, a mixed signal from the purple stimulus produces:
purple=np.array([0,0,0,1,0,1,0,0,0])
purple_act=cor_mat_nonuniform@purple.T

#This section of code visualizes the output

purple_act_im=np.zeros((sizeF,sizeF))
np.fill_diagonal(purple_act_im,purple_act)
ub=np.amax(np.amax(np.abs(purple_act_im)))
lb=-ub


plt.figure()
plt.stem(purple_act)
plt.title('Activation from Purple Stimulus with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero

plt.figure()
plt.imshow(purple_act_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Activation from Purple Stimulus with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()

# To interpret the output, remember the feature list:
# FeatureNames=['small','medium','large','red','yellow','blue','circle','rectangle','triangle']


### Exercise 3
Consider the code in the following cell.  Your task is to run the cell and reverse engineer what it is accomplishing.  Add comments to each part of the code using # to explain what it is doing.  Then write a few sentences about the entire cell and what useful function it is performing.   

In [ ]:
scalar=0
for i in range (3):
    scalar=scalar+cor_mat_nonuniform[i,i]**2
scalar=scalar**(-1/2)

cor_mat_nonuniform=scalar*cor_mat_nonuniform
ub=np.amax(np.amax(np.abs(cor_mat_nonuniform)))
lb=-ub

# This portion of the code visualizes the correlation matrix
plt.figure()
plt.imshow(cor_mat_nonuniform,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Correlation Matrix Created by Matrix Multiplication for Nonuniform Stimuli')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()


purple_act=cor_mat_nonuniform@purple.T
#This section of code visualizes the output

purple_act_im=np.zeros((sizeF,sizeF))
ub=np.amax(np.amax(np.abs(cor_mat_command)))
lb=-ub
np.fill_diagonal(purple_act_im,purple_act)

plt.figure()
plt.stem(purple_act)
plt.title('Activation from Purple Stimulus with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.xlabel('Feature')
plt.ylabel('Activation')
plt.show()

# For this image, the vector is mapped to the diagonal of the image and the
# off-diagonal values are set to zero

plt.figure()
plt.imshow(purple_act_im,cmap='RdBu_r',vmin=lb,vmax=ub)
plt.title('Activation from Purple Stimulus with Cor_mat_nonuniform')
plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)
plt.colorbar()
plt.show()


## Self-correlation

Up until this point, we have been creating correlation matrices that include self-correlation. That is, a stimulus is positively correlated with itself. This makes sense, because all stimuli correlate with themselves. That is, if we are shown something red, we know we have seen something red. However, if we consider what an output of a Hebbian system should be: the stimuli the learner *associates* with an input (I see red, I think stop sign), this doesn't seem quite right.  After all, if one were prompted to respond to the word "red" with the first word that comes to mind, one would not typically respond, "red".  

From another perspective, a neuron with positive excitatory connections to itself would behave a bit like a microphone placed next to a speaker that is amplifying the input to the microphone -- a runaway positive feedback loop.  In most cases we don't want our models to have seizures.   
  
It follows, that when constructing correlation matrices, it could be useful to remove self-correlation. 

### Exercise 4: Remove self-correlation 
Create a correlation matrix that does not include self-correlation.

## Hebbian Learning In PsyNeuLink

Hebbian learning is achieved in PsyNeuLink by creating an appropriately sized recurrent transfer mechanism, and enabling learning. To do this, when defining your transfer mechanism, we simply set the argument "enable_learning" to True (always include the capital T).

To initialize your matrix to zero, as we have been doing, set the arguments "auto" to 0 and "hetero" to 0.

### Hebbian Model #3

In [ ]:
#sample Hebb

Hebb_comp = pnl.Composition()

Hebb_mech=pnl.RecurrentTransferMechanism(
    size=sizeF,
    function=pnl.Linear,
    #integrator_mode = True,
    #integration_rate = 0.5,
    enable_learning = True,
    #learning_rate = 0.5,
    name='Hebb_mech',
    #matrix=pnl.AutoAssociativeProjection,
    auto=0,
    hetero=0
    )

Hebb_comp.add_node(Hebb_mech)


Let's try training our new system on a single stimulus from the set we've been working with; small red circle. 

We can run our system as many times as we like, using only one stimulus as input. In the following cell, we run the system 5 times (num_trials = 5).   

Does the system behave the way you expected it to? In what ways does it behave similar and different to the previous two Hebbian Models #1 & #2?

In [ ]:
Hebb_comp.execution_id = 1

# Use print_info to show numerical values and vis_info to show graphs of the changing values

def print_info():
    print('\nWeight matrix:\n', Hebb_mech.matrix.base, '\nActivity: ', Hebb_mech.value)

    
def vis_info():
  hebb_matrix = Hebb_mech.matrix.base
  ub=np.amax(np.amax(np.abs(hebb_matrix)))
  lb=-ub
  plt.figure()
  plt.imshow(hebb_matrix,cmap='RdBu_r',vmin=lb,vmax=ub)
  plt.title('PNL Hebbian Matrix')
  plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
  plt.yticks(np.arange(0,sizeF),FeatureNames,rotation=35)

  plt.colorbar()
  plt.show()

  plt.figure()
  plt.stem(Hebb_mech.value[0])
  plt.title('Activation from Stimulus with PNL Hebbian Matrix')
  plt.xticks(np.arange(0,sizeF),FeatureNames,rotation=35)
  plt.xlabel('Feature')
  plt.ylabel('Activation')
  plt.show()

inputs_dict = {Hebb_mech:np.array(src)}
out=Hebb_comp.learn(num_trials=5,
      call_after_trial=vis_info,
      inputs=inputs_dict)

To better understand what is happening, try running your system once with your chosen input (e.g. src), and then several more times with no input (all 0's).  To do this you will need to re-initialize the system by rerunning the cell that created Hebbian Model #3, or by copying that code into the top of the next cell.  Models that learn get modified by running, and so we need to re-initialize to see how the model evolves with different kinds of input.    

# Exercise 5:

Write an explanation of what is happening, mathematically and qualitatively in the PNL Hebbian algorithm. If your understanding is correct, you will understand what occurs in the next section

# Orthogonal stimuli in PNL:

Now, we are going to train our system on two unrelated stimuli, src and myr. Use your previous code to train the system 5 times on src and 5 times on myr. 

Does the system behave as expected?

If we wanted to produce results similar to our earlier Hebbian models, how would we do so? Alter the code so that it behaves that way. (Hint: Look for a relation between the weight matrix and the previous _output_ of the network. How could you force the output to be zero?)

# Bonus Exercises

Refer to the homework formatting and submission instructions from Lab 1.   Also, for Lab 2 be sure to include your answers to the other exercises from throughout the lab.


## Exercise 6: Stump the Hebbian Learning Algorithm
Create a set of at least 9 unique stimuli using the same 9 feautres as above with properties that are impossible for the Hebbian Model #1 to learn and use for accurate inferences.  Even after exposure to the stimuli, the correlation matrix should be full of 0s.  Explain why these results are produced.  Try to come up with an interesting example that might map on to a real world situation (rather than just a mathematical gimic).  


## Exercise 7: Pruning Instead of Connecting

Build a new model based on Hebbian Model #2 that starts with every unit connected to every other neuron, all with equal positive weights (all 1s).  As this model receives stimuli it should preserve the connections between neurons that fire together, and gradually weaken the connections between neurons that do not fire together.  

A prominent feature of brain development from infancy to adulthood is the reduction of neurons and increase in glial cells (Burek & Openheim, 1996; Low & Cheng, 2006).  For example, one study inspecting a brain area for comparison between newborns and adults found that the adult brains had about 40% fewer neurons and 340% more glials cells (Abitz et al, 2007).  Weakening and elimination of connections is often called pruning, and might be more common than growing entirely new connections.  Imagine that a neuron somewhere near your forehead is firing correlated with a neuron somewhere near the back of your head -- it would be difficult to grow an entirely new axon that connects the two.  Instead, during early development when the brain begins by occupying a tiny volume, it is easy to start with many neurons connected and preserve these connections as the brain grows larger.  Later, neurons that are connected and fire together can strengthen their connections, while neurons that fire independently can weaken their connections or let the connections be pruned away entirely. 

Train this model on stim_mat and test it to make inferences on at least three different single-feature stimuli, such as triangle = [0,0,0,0,0,0,0,0,1].  How does it perform?  

Discuss the various choices that you made when creating this model.  For example, do you allow the connection weights between neurons to ever go below 0?  Why or why not?  What rate of pruning did you decide to implement, and why did you choose this number?  How might you reasonably adjust the pruning rate to depend on the total number of expected stimuli (would you use the same rate for 3 stimuli and 3000 stimuli)?   


## Exercise 8:  Both Connecting & Pruning

### 8a.  Combined Model
Combine Hebbian Model #2 with your model created above for Homework #2a to create a more sophisticated model that both strengthens connections between neurons that fire together and prunes away connections between neurons that fire independently.  Use stim_mat to train your model.  You will need to find a good balance between the rate of growing (strengthening) connections and the rate of pruning (weakening) connections to ensure that the model learns effectively.  Explain how these rates interact and discuss the growing & pruning rates that appear to optimize learning over the current set of stimuli.      

### 8b.  Complex Stimuli
Add 6 new complex stimuli to the learning set.  Each new stimulus should activate exactly 5 feature neurons.  See the example of "small green house" in the cell introducing the stimuli.  Be sensible in how you combine features: medium+small has an obvious interpretation between medium and small, but small+large does not.  Your new set of stimuli will contain the original 3 (src, myr, lbt) plus the 6 new objects you specify.  Train the model you created for #3a above, and then test it to make inferences on partial stimuli (e.g. triangle = [0,0,0,0,0,0,0,0,1]).  Try to optimize the rates of connection growth and pruning to handle this more complex set of stimuli, and explain what you tried and what you learned.  How do you interpret the inferences made when testing this model?     

## Exercise 9:  PsyNeuLink Hebbian Necker Cube
Build a model that uses Hebbian learning in PsyNeuLink to discover the two distinct interpretations of the Necker Cube.  For stimuli, this model will get exposed to one corner of a consistent interpretation of the cube at a time. The cell after the BUILD prompt will help you get started.  It creates a set of 16 stimuli that will be the input to your model.     




In [ ]:
Build_N_Matrix = np.zeros((16,5))
Build_N_Matrix[0,:] = [0, 1, 3, 4, 8]
Build_N_Matrix[1,:] = [1, 0, 2, 5, 9]
Build_N_Matrix[2,:] = [2, 1, 3, 6, 10]
Build_N_Matrix[3,:] = [3, 0, 2, 7, 11]
Build_N_Matrix[4,:] = [4, 5, 7, 0, 12]
Build_N_Matrix[5,:] = [5, 4, 6, 1, 13]
Build_N_Matrix[6,:] = [6, 5, 7, 2, 14]
Build_N_Matrix[7,:] = [7, 4, 6, 3, 15]
Build_N_Matrix[8,:] = [8, 9, 11, 12, 0]
Build_N_Matrix[9,:] = [9, 8, 10, 13, 1]
Build_N_Matrix[10,:] = [10, 9, 11, 14, 2]
Build_N_Matrix[11,:] = [11, 8, 10, 15, 3]
Build_N_Matrix[12,:] = [12, 13, 15, 8, 4]
Build_N_Matrix[13,:] = [13, 12, 14, 9, 5]
Build_N_Matrix[14,:] = [14, 13, 15, 10, 6]
Build_N_Matrix[15,:] = [15, 12, 14, 11, 7]

Build_N_Matrix = Build_N_Matrix.astype(int)

Necker_Matrix = np.zeros((16,16))
Necker_Matrix = Necker_Matrix.astype(int)

excite = 1

for x in range(0,16):
    Necker_Matrix[Build_N_Matrix[x,0], Build_N_Matrix[x,0]] = excite
    Necker_Matrix[Build_N_Matrix[x,0], Build_N_Matrix[x,1]] = excite
    Necker_Matrix[Build_N_Matrix[x,0], Build_N_Matrix[x,2]] = excite
    Necker_Matrix[Build_N_Matrix[x,0], Build_N_Matrix[x,3]] = excite

# The following line of code should be run after you initialize and name your RecurrentTransferMechanism.  
# It breaks the Necker_Matrix defined above into 16 arrays that will be a dictionary of inputs. 
# You need to replace "Hebb" below with the name of your RecurrentTransferMechanism that receives input.  
input_dictNC = {Hebb: np.split(Necker_Matrix,16)}

input_dictNC